In [1]:
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D, AveragePooling2D
from keras import regularizers

from tensorflow.keras.optimizers.schedules import ExponentialDecay
from tensorflow.keras.optimizers import Adam

import numpy as np
import cv2

import os
import random
import pickle
import time

Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.
Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.


In [2]:
pickle_X_in = open("X.pickle", "rb")
X = pickle.load(pickle_X_in)

pickle_y_in = open("y.pickle", "rb")
y = pickle.load(pickle_y_in)

X = X/255.0

X_train, X_val,y_train, y_val = train_test_split(X, y, test_size=0.1, random_state=241)
y_train = np.array(y_train)
y_val = np.array(y_val)

## Find the best stride window

In [3]:
strides = [1, 2, 3, 4, 5, 6]
for stride in strides:

    name = "{}-stride-{}".format(stride, int(time.time()))
    model = Sequential([
                Conv2D(128, (stride, stride), activation="tanh", kernel_regularizer=regularizers.l2(0.01)),
                AveragePooling2D(pool_size=(3, 3)),
                Conv2D(64, (stride, stride), activation="tanh", kernel_regularizer=regularizers.l2(0.01)),
                AveragePooling2D(pool_size=(stride, stride)),
                Flatten(),
                Dense(64, activation="tanh"),
                Dense(1, activation="sigmoid")
            ])

    learning_rate_schedule = ExponentialDecay( 0.0001, 1500, 0.8 )

    opt = Adam(learning_rate=learning_rate_schedule)

    model.compile(
        loss='binary_crossentropy',
        optimizer=opt,
        metrics = ['accuracy']
    )
    tensorboard = tf.keras.callbacks.TensorBoard(log_dir="strides_logs/{}".format(name))
    history = model.fit(X_train, y_train, batch_size=32, epochs=5, validation_data=(X_val, y_val), shuffle=True,callbacks=[tensorboard])

Epoch 1/5
6/6 [==============================] - 9s 967ms/step - loss: 1.8409 - accuracy: 0.5284 - val_loss: 1.3986 - val_accuracy: 0.6500
Epoch 2/5
6/6 [==============================] - 5s 837ms/step - loss: 1.6554 - accuracy: 0.6534 - val_loss: 1.4703 - val_accuracy: 0.6000
Epoch 3/5
6/6 [==============================] - 5s 906ms/step - loss: 1.4578 - accuracy: 0.7386 - val_loss: 1.4061 - val_accuracy: 0.8000
Epoch 4/5
6/6 [==============================] - 6s 953ms/step - loss: 1.3769 - accuracy: 0.7898 - val_loss: 1.6088 - val_accuracy: 0.5500
Epoch 5/5
6/6 [==============================] - 6s 925ms/step - loss: 1.3900 - accuracy: 0.7727 - val_loss: 1.3962 - val_accuracy: 0.6500
Epoch 1/5
6/6 [==============================] - 8s 966ms/step - loss: 1.5307 - accuracy: 0.5739 - val_loss: 1.4482 - val_accuracy: 0.8000
Epoch 2/5
6/6 [==============================] - 5s 904ms/step - loss: 1.4625 - accuracy: 0.7273 - val_loss: 1.4342 - val_accuracy: 0.6000
Epoch 3/5
6/6 [============

## Conclusion

In [1]:
'''
After examining the graphs provided by Tensorboard, we find that the model performs best with
a (2, 2) stride window
'''

'\nAfter examining the graphs provided by Tensorboard, we find that the model performs best with\na (2, 2) stride window\n'